### Use Antagonist to train a symptom detection model 

#### Reproducibility

In [ ]:
# Torch
import torch
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

# Python
import random
random.seed(0)

# Numpy
import numpy as np
np.random.seed(0)

#### Dataset preparation

Note: the dataset needs to be downloaded using the script `download_SMD_dataset.sh` in the `scripts/antagonist-ml` folder.

In [ ]:
from data_utils import SMD

In [ ]:
db = SMD(dataset_folder=r"D:\antagonist\data\ServerMachineDataset")

In [ ]:
dataframes, files = db.read_dataset(group_name="Group 1", train=False, retrieve_labels=True)


In [ ]:
service_idx = 0
df,service_id = dataframes[service_idx], files[service_idx]
df,labels = df[df.columns[:-2].tolist()+['timestamp']], df[['label']]

In [ ]:
network_incidents = db.get_interpretation_labels(service_id)

In [ ]:
network_incidents

#### Train model utils

In [ ]:
import pandas as pd
from auto_encoder import Vanilla_AE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score

In [ ]:
n_inputs = df.shape[1] - 1
layer_sizes = [8, 4, 2]
lr = 0.005
batch_size = 32
epochs = 40
validation_split = 0.2
early_stopping = True
patience = 3
Q = 0.99  # residual cut

In [ ]:
def train_model(df: pd.DataFrame):
    ae = Vanilla_AE(n_inputs=n_inputs, layer_sizes=layer_sizes)

    # Get data but the timestamp
    X_train = df.values[:, :-1]

    # scaler init and fitting
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # model fitting
    ae.fit(
        X_train_scaled,
        early_stopping=early_stopping,
        validation_split=validation_split,
        epochs=epochs,
        lr=lr,
        batch_size=batch_size,
        verbose=0,
        shuffle=True,
        patience=patience,
        delta=0.001,
    )

    # results predicting
    residuals_train = (
        pd.DataFrame(X_train_scaled - ae.predict(X_train_scaled)).abs().sum(axis=1)
    )
    threshold = residuals_train.quantile(Q) * 5 / 2

    return scaler, ae, threshold 

In [ ]:
def eval_model(df, labels,  scaler, ae, threshold ):

    X_hat = scaler.transform(df.values[:,:-1])
    residuals_full_df = X_hat - ae.predict(X_hat)
    residuals_full_df = pd.DataFrame(residuals_full_df).abs().sum(axis=1).to_frame()
    residuals_full_df['outlier'] = (residuals_full_df > threshold).astype(int).values

    return f1_score(labels['label'].values,residuals_full_df['outlier'].values, average='binary')

#### Iterative process simulation

Every day the model is retrained with the new data and new labels

In [ ]:
import datetime


In [ ]:
start_date = datetime.datetime.fromtimestamp(df['timestamp'].astype('int64').min()/10**9)
end_date = datetime.datetime.fromtimestamp(df['timestamp'].dt.ceil('D').astype('int64').max()/10**9)

In [ ]:
scaler, ae, threshold = None, None, None
for current_day in pd.date_range(start=start_date, end=end_date, freq="D"):
    current_day = datetime.datetime.fromtimestamp(current_day.timestamp())
    df_today = df.loc[df["timestamp"] < current_day.ctime()]

    if df_today.shape[0] == 0:
        # first day
        continue

    if scaler is not None:
        f1 = eval_model(df_today, labels[: df_today.shape[0]], scaler, ae, threshold)
        print(f"F1 score from last day: {round(f1,4)}")
    
    scaler, ae, threshold = train_model(df_today)
        

In [ ]:
f1 = eval_model(df, labels, scaler, ae, threshold)
print(f"F1 score: {round(f1,4)}")